# SLEEP POSTURE MONITORING



In [ ]:
# Libraries
import sys
import pickle
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sas
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn.preprocessing as pre
from numpy import linalg as la
from sklearn.cluster import KMeans
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix

**PREPROCESSING FUNCTIONS**

In [ ]:
def PCA(file):
    df = pd.read_csv(file, sep="\t", header=None)
    df.drop(columns=[2048], inplace = True) 
    df.drop(0, inplace = True) 
    df.drop(1, inplace = True)
    df.drop(2, inplace = True)
    
    array = df.to_numpy()
    avg = np.mean(array, axis = 1, keepdims=True)

    array = array - avg

    CX = np.cov(array)
    w,v = la.eig(CX) 

    CY =  np.diag(np.round(np.real(w),2))
    
    tr = 0;
    for i in range(0,5):
        tr = tr + CY[i,i]
        print("Fraction of the total variability keeping", i+1,"the first principal component: ", np.round(100*tr/CY.trace()),"%")

    P = v[:,0:1]
    P = abs(P)

    array_final = P.T.dot(array)
        
    return array_final

def rescale(pressure_map, max_value, min_value):  
    maximum = pressure_map.max()
    minimum = pressure_map.min()  
    OldRange = maximum - minimum
    NewRange = max_value - min_value

    for i in range(0,len(pressure_map)):
        pressure_map[i] = (((pressure_map[i] - minimum) * NewRange) / OldRange) 
            
    return pressure_map

def kMeans(section):
    X = []
    
    for i in range(0, section.shape[0]):
        for j in range(0, section.shape[1]):
            t = []
            t.append(i)
            t.append(j)
            t.append(section[i][j])
            X.append(t)

    X = np.array(X)

    kmeans = KMeans(n_clusters=2).fit(X)
    
    # compute x, y centroid and avg intensity in cluster
    cluster_1 = []
    cluster_2 = []

    for i in range(0,len(X)):
        if kmeans.labels_[i] == 0:
            cluster_1.append(X[i])
        else:
            cluster_2.append(X[i])

    mean_1 = np.mean(cluster_1, axis = 0) 
    mean_2 = np.mean(cluster_2, axis = 0) 

    if mean_1[2] > mean_2[2]:
        mean_cluster = mean_1[2]
        y_pos = kmeans.cluster_centers_[0][0]
        x_pos = kmeans.cluster_centers_[0][1]
    else:
        mean_cluster = mean_2[2]
        y_pos = kmeans.cluster_centers_[1][0]
        x_pos = kmeans.cluster_centers_[1][1]

    
    return x_pos, y_pos, mean_cluster, kmeans.labels_


**IMPORTING AND PLOTTING DATA**

In [ ]:
#collecting some examples of data from the first part of the dataset
data_folder = Path("dataset/experiment-i/S3/")

file_name = []
array = []

file_name.append(data_folder / "1.txt")
file_name.append(data_folder / "2.txt")
file_name.append(data_folder / "3.txt")

for i in range(0,len(file_name)):
    df = pd.read_csv(file_name[i], sep="\t", header=None)
    df.drop(columns=[2048], inplace = True) 
    df.drop(0, inplace = True) 
    df.drop(1, inplace = True) 
    df.drop(2, inplace = True) 
    
    array.append(df.to_numpy())


#plotting one sample for each imported file 
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10,5))
fig.suptitle('Example of one sample for each positions')
g1 = sas.heatmap(np.flip(array[0][1].reshape(64,32),1), cmap="viridis", cbar=False, ax=ax1, xticklabels=8, yticklabels=16)
g2 = sas.heatmap(np.flip(array[1][1].reshape(64,32),1), cmap="viridis", cbar=False, ax=ax2, xticklabels=8, yticklabels=16)
g3 = sas.heatmap(np.flip(array[2][1].reshape(64,32),1), cmap="viridis", ax=ax3, xticklabels=8, yticklabels=16)
plt.tight_layout()
plt.show()


**PCA RESULTS**

In [ ]:
PCA_result = []
for i in range(0, len(file_name)):

    print("POSITION", i, "\n")
    tmp = PCA(file_name[i])
    PCA_result.append(rescale(tmp, 700, 0))
    print("\n")
    

In [ ]:
#plot the results of the PCA 
fig, ax = plt.subplots(2, 3, figsize=(10,10))
fig.suptitle('Compare PCA projection with original sample')
g1 = sas.heatmap(np.flip(array[0][1].reshape(64,32),1),cmap="viridis",cbar=False,ax=ax[0,0], xticklabels=8, yticklabels=16)
g2 = sas.heatmap(np.flip(array[1][1].reshape(64,32),1),cmap="viridis",cbar=False,ax=ax[0,1], xticklabels=8, yticklabels=16)
g3 = sas.heatmap(np.flip(array[2][1].reshape(64,32),1),cmap="viridis",ax=ax[0,2], xticklabels=8, yticklabels=16)
g4 = sas.heatmap(np.flip(PCA_result[0].reshape(64,32),1),cmap="viridis",cbar=False,ax=ax[1,0], xticklabels=8, yticklabels=16)
g5 = sas.heatmap(np.flip(PCA_result[1].reshape(64,32),1),cmap="viridis",cbar=False,ax=ax[1,1], xticklabels=8, yticklabels=16)
g6 = sas.heatmap(np.flip(PCA_result[2].reshape(64,32),1),cmap="viridis",ax=ax[1,2], xticklabels=8, yticklabels=16)
plt.tight_layout()
plt.show()


In [ ]:
#collecting some examples of data from the second part of the dataset
data_folder = Path("dataset/experiment-ii/S3/Air_Mat/")

file_name = []
array_2 = []

file_name.append(data_folder / "Matrix_Air_B3.txt")
file_name.append(data_folder / "Matrix_Air_D1.txt")
file_name.append(data_folder / "Matrix_Air_C2.txt")

for i in range(0,len(file_name)):
    df = pd.read_csv(file_name[i], sep=" ", header=None)
    
    array_2.append(df.to_numpy())

#plotting one sample for each imported file
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(10,5))
fig.suptitle('Example of positions of the second part of the datatset')
g1 = sas.heatmap(np.flip(array_2[0].reshape(64,27),0),cmap="viridis",cbar=False,ax=ax1, xticklabels=8, yticklabels=16)
g2 = sas.heatmap(np.flip(array_2[1].reshape(64,27),0),cmap="viridis",cbar=False,ax=ax2, xticklabels=8, yticklabels=16)
g3 = sas.heatmap(np.flip(array_2[2].reshape(64,27),0),cmap="viridis",ax=ax3, xticklabels=8, yticklabels=16)
plt.tight_layout()
plt.show()


**EXPERIMENT 1**

**DATASET**

In [ ]:
dataset_exp1 = pd.read_csv('dataset.csv')
dataset_exp1 = dataset_exp1.sample(frac=1).reset_index(drop=True)
dataset_exp1


In [ ]:
#load the model
model_position = load_model('Model/model_position.h5')
model_subject = load_model('Model/model_subject.h5')

#summarize model
print('POSITION MODEL SUMMARY','\n')
model_position.summary()
print()
print('SUBJECT MODEL SUMMARY','\n')
model_subject.summary()

with open('Model/test.pkl', 'rb') as f:
    [test_data, test_subject, test_position, test_labels_s, test_labels_p] = pickle.load(f)
    

In [ ]:
print('EVALUATION POSITIONS')
model_position.evaluate(test_data, test_position)

print()
print('EVALUATION SUBJECTS')
model_subject.evaluate(test_data, test_subject)

print()

In [ ]:
predictions_subject = model_subject.predict(test_data)
predictions_position = model_position.predict(test_data)

predictions_subject = predictions_subject.argmax(axis=-1)
predictions_position = predictions_position.argmax(axis=-1)

plt.figure(figsize=(20,10))

for i in range(1, 11):
    plt.subplot(2,5,i)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    
    label = str(predictions_subject[i-1]) + ' - ' + str(predictions_position[i-1])
    plt.imshow(test_data[i-1].reshape(64, 32), cmap='gray')
    plt.xlabel(label)

In [ ]:
labels = ['Supine', 'Right', 'Left']
cm = confusion_matrix(test_labels_p, predictions_position, [0,1,2])

f,ax= plt.subplots(figsize=(10,10))
sas.heatmap(cm, annot=True, ax = ax, linewidths=1, fmt = 'd', cmap = "Blues")
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix Position')
ax.xaxis.set_ticklabels(labels); ax.yaxis.set_ticklabels(labels)

plt.show()


In [ ]:
labels = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6','S7', 'S8', 'S9','S10', 'S11', 'S12','S13']

cm = confusion_matrix(test_labels_s, predictions_subject, [0,1,2,3,4,5,6,7,8,9,10,11,12])

f,ax= plt.subplots(figsize=(10,10))
sas.heatmap(cm, annot=True, ax = ax, linewidths=1, fmt = 'd', cmap="Greens")
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix Subject')
ax.xaxis.set_ticklabels(labels)
ax.yaxis.set_ticklabels(labels)

plt.show()


**EXPERIMENT 2**

**DATASET**

In [ ]:
dataset_exp2 = pd.read_csv('dataset_2.csv', sep=",", header=0)
dataset_exp2 = dataset_exp2.sample(frac=1).reset_index(drop=True)
dataset_exp2

**K-MEANS RESULTS**

In [ ]:
fig, ax = plt.subplots(3, 4, figsize=(10,10))

for i in range(0,len(PCA_result)):
    tmp = np.flip(PCA_result[i].reshape(64,32),1)
    section1 = tmp[:14]
    section2 = tmp[14:37]
    section3 = tmp[37:]
    pos_h_x, pos_h_y, mean_h, labels_1 = kMeans(section1)
    pos_c_x, pos_c_y, mean_c, labels_2 = kMeans(section2)
    pos_l_x, pos_l_y, mean_l, labels_3 = kMeans(section3)
    
    tot = np.concatenate((labels_1, labels_2, labels_3), axis=0)
    
    g1 = sas.heatmap(labels_1.reshape(14,32),cmap="Greens",cbar=False,ax=ax[i,0], xticklabels=7, yticklabels=16)
    #ax[i,0].scatter(pos_h_x, pos_h_y, marker='o', s=100, linewidths=3, color='g', zorder=10)
    g2 = sas.heatmap(labels_2.reshape(23,32),cmap="Greens",cbar=False,ax=ax[i,1], xticklabels=8, yticklabels=16)
    #ax[i,1].scatter(pos_c_x, pos_c_y, marker='o', s=100, linewidths=3, color='g', zorder=10)
    g3 = sas.heatmap(labels_3.reshape(27,32),cmap="Greens",cbar=False,ax=ax[i,2], xticklabels=8, yticklabels=16)
    #ax[i,2].scatter(pos_l_x, pos_l_y, marker='o', s=100, linewidths=3, color='g', zorder=10)
    g4 = sas.heatmap(tot.reshape(64,32),cmap="Greens",cbar=False,ax=ax[i,3], xticklabels=8, yticklabels=16)
    ax[i,3].scatter(pos_h_x, pos_h_y, marker='o', s=100, linewidths=3, color='b', zorder=10)
    ax[i,3].scatter(pos_c_x, pos_c_y + 14, marker='o', s=100, linewidths=3, color='b', zorder=10)
    ax[i,3].scatter(pos_l_x, pos_l_y + 37, marker='o', s=100, linewidths=3, color='b', zorder=10)
    
plt.tight_layout()
plt.show()


In [ ]:
#load the model
model_position = load_model('Model/model_position_2.h5')
model_subject = load_model('Model/model_subject_2.h5')

#summarize model
print('POSITION MODEL SUMMARY','\n')
model_position.summary()
print()
print('SUBJECT MODEL SUMMARY','\n')
model_subject.summary()

with open('Model/test_2.pkl', 'rb') as f:
    [test_data_s, test_data_p, test_subject, test_position, test_labels_s, test_labels_p] = pickle.load(f)
    

In [ ]:
print('EVALUATION POSITIONS')
model_position.evaluate(test_data_p, test_position)

print()
print('EVALUATION SUBJECTS')
model_subject.evaluate(test_data_s, test_subject)

print()

In [ ]:
predictions_position = model_position.predict(test_data_p)
predictions_position = predictions_position.argmax(axis=-1)

predictions_subject = model_subject.predict(test_data_s)
predictions_subject = predictions_subject.argmax(axis=-1)


In [ ]:
labels = ['Supine', 'Right', 'Left']

cm = confusion_matrix(test_labels_p, predictions_position, [0,1,2])

f,ax= plt.subplots(figsize=(10,10))
sas.heatmap(cm, annot=True, ax = ax, linewidths=1, fmt = 'd', cmap = "Blues")
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix Position')
ax.xaxis.set_ticklabels(labels)
ax.yaxis.set_ticklabels(labels)

plt.show()


In [ ]:
labels = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6','S7', 'S8', 'S9','S10', 'S11', 'S12','S13']

cm = confusion_matrix(test_labels_s, predictions_subject, [0,1,2,3,4,5,6,7,8,9,10,11,12])

f,ax= plt.subplots(figsize=(10,10))
sas.heatmap(cm, annot=True, ax = ax, linewidths=1, fmt = 'd', cmap="Greens") 
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)

# labels, title and ticks
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix Subject') 
ax.xaxis.set_ticklabels(labels)
ax.yaxis.set_ticklabels(labels)

plt.show()
